In [1]:
# ------------------------------------------------------------
# Configuración Inicial del Entorno
# ------------------------------------------------------------
import sys
import os
import pprint
import torch

# Ruta base del proyecto (sube un nivel desde el directorio actual)
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
SRC_PATH = os.path.join(BASE_DIR, "src")

# Verificar si `src` está en sys.path, si no, agregarlo
if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)
    print(f"'src' agregado a sys.path: {SRC_PATH}")

# Verificar que la carpeta `loaders` exista dentro de `src`
loaders_path = os.path.join(SRC_PATH, "loaders")
if not os.path.exists(loaders_path):
    raise FileNotFoundError(f"No se encontró la carpeta 'loaders' en {SRC_PATH}. Verifica la estructura del proyecto.")

'src' agregado a sys.path: g:\Mi unidad\EnorChile\GIT\practicos-rag\src


In [2]:
# ------------------------------------------------------------
# Importar módulos necesarios
# ------------------------------------------------------------
try:
    from loaders.pdf_processor import process_pdf_directly
    from loaders.table_review import process_and_review_tables
    from loaders.pdf_inspector import inspect_pdf_content
    print("Módulos importados correctamente.")
except ModuleNotFoundError as e:
    print(f"Error al importar módulos: {e}")
    print("--- Rutas actuales en sys.path ---")
    for path in sys.path:
        print(path)
    raise

Módulos importados correctamente.


In [3]:
import importlib.util

# Cargar el módulo `reranking` manualmente
reranking_path = os.path.join(SRC_PATH, "retrievers", "reranking.py")
spec = importlib.util.spec_from_file_location("retrievers.reranking", reranking_path)
reranking = importlib.util.module_from_spec(spec)
spec.loader.exec_module(reranking)

# Cargar el módulo `hybrid_search` manualmente
hybrid_search_path = os.path.join(SRC_PATH, "retrievers", "hybrid_search.py")
spec = importlib.util.spec_from_file_location("retrievers.hybrid_search", hybrid_search_path)
hybrid_search = importlib.util.module_from_spec(spec)
spec.loader.exec_module(hybrid_search)

# Asignar funciones y modelos a variables
embedding_model = reranking.embedding_model
generate_keywords_with_t5 = reranking.generate_keywords_with_t5
expand_keywords_with_similarity = reranking.expand_keywords_with_similarity
hybrid_reranking = hybrid_search.hybrid_reranking

print("Importaciones manuales de `retrievers` completadas.")

c:\Users\gigle\anaconda3\envs\mia\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Importaciones manuales de `retrievers` completadas.


In [4]:
# ------------------------------------------------------------
# Conectar a Qdrant
# ------------------------------------------------------------
from dotenv import load_dotenv
import os
from qdrant_client import QdrantClient

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Leer las variables desde .env
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
INDEX_NAME = "embeddings-version-2"

# Validar que las variables se cargaron correctamente
if not QDRANT_URL or not QDRANT_API_KEY:
    raise ValueError("Las variables QDRANT_URL o QDRANT_API_KEY no están configuradas correctamente en el archivo .env")

print("Conectando a Qdrant...")
qdrant_client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)
print("Conexión a Qdrant establecida con éxito.")

Conectando a Qdrant...
Conexión a Qdrant establecida con éxito.


# Preprocessing

In [ ]:
# ------------------------------------------------------------
# Configurar las rutas de trabajo y mostrar detalles de los PDFs
# ------------------------------------------------------------
import pdfplumber

print("\n--- Configurando las rutas de trabajo ---")
CAPITULOS_DIR = os.path.join(BASE_DIR, "data/capitulos")
if not os.path.exists(CAPITULOS_DIR):
    raise FileNotFoundError(f"No se encontró el directorio 'data/capitulos' en {BASE_DIR}.")

# Obtener todos los PDFs en el directorio
pdf_paths = [
    os.path.join(CAPITULOS_DIR, file)
    for file in os.listdir(CAPITULOS_DIR)
    if file.endswith(".pdf")
]

# Mostrar resumen inicial de los archivos PDF y su cantidad de páginas
print(f"\n--- {len(pdf_paths)} PDFs encontrados en '{CAPITULOS_DIR}' ---")
for idx, pdf_path in enumerate(pdf_paths, start=1):
    try:
        with pdfplumber.open(pdf_path) as pdf:
            page_count = len(pdf.pages)
            print(f"{idx}. {os.path.basename(pdf_path)} - {page_count} páginas")
    except Exception as e:
        print(f"{idx}. {os.path.basename(pdf_path)} - Error al contar páginas: {e}")

In [ ]:
# ------------------------------------------------------------
# Procesar PDFs directamente
# ------------------------------------------------------------
print("\n--- Procesando TODOS los PDFs en el directorio ---")
results = process_pdf_directly(pdf_paths)

# Mostrar resultados del procesamiento
print("\n--- Resumen del procesamiento ---")
for pdf, data in results.items():
    print(f"{os.path.basename(pdf)}: {data['total_paginas']} páginas, {data['total_bloques']} bloques de texto, {data['total_tablas']} tablas.")

In [ ]:
# ------------------------------------------------------------
# Revisar tablas en documentos seleccionados y limitar cantidad
# ------------------------------------------------------------

# Lista de documentos a revisar y el número máximo de tablas por documento
documents_to_review = {
    "capitulo7.pdf": 3,  # Revisar hasta 3 tablas del documento capitulo7.pdf
    "capitulo14.pdf": 3,  # Revisar hasta 3 tablas del documento capitulo14.pdf
    "capitulo19.pdf": 3,  # Revisar hasta 3 tablas del documento capitulo19.pdf
    "capitulo22.pdf": 2,  # Revisar hasta 2 tablas del documento capitulo22.pdf
}

print("\n--- Revisando tablas en los documentos seleccionados ---")

# Iterar sobre cada documento y su límite de tablas
for pdf_name, max_tables in documents_to_review.items():
    # Construir la ruta completa del documento
    pdf_file = os.path.join(CAPITULOS_DIR, pdf_name)
    
    # Verificar si el archivo existe
    if not os.path.exists(pdf_file):
        print(f"Archivo no encontrado: {pdf_name}, saltando...")
        continue

    print(f"\nRevisando tablas en {pdf_name} (límite: {max_tables} tablas)")

    try:
        # Llamar a la función con el límite de tablas especificado
        tables = process_and_review_tables(pdf_file, max_tables=max_tables)
        
        # Mostrar el resultado del procesamiento
        print(f"Se procesaron {len(tables)} tablas en {pdf_name} (límite era {max_tables})")
    except Exception as e:
        # Capturar y mostrar cualquier error
        print(f"Error al procesar {pdf_name}: {e}")
    
    print("\n" + "="*80 + "\n")

In [ ]:
# ------------------------------------------------------------
# Inspeccionar contenido de los PDFs con un límite de caracteres
# ------------------------------------------------------------
print("\n--- Inspeccionando contenido de los PDFs con límite de caracteres ---")
inspect_pdf_content(CAPITULOS_DIR, pdf_paths, num_pages_limit=5, char_limit=500)

In [ ]:
from loaders.metadata_processor import process_pdfs_with_validated_metadata

# ------------------------------------------------------------
# Procesar los PDFs y generar datos enriquecidos
# ------------------------------------------------------------
print("\n--- Procesando PDFs y generando datos enriquecidos ---")
data_enriched = process_pdfs_with_validated_metadata(pdf_paths, verbose=False)

In [ ]:
# ------------------------------------------------------------
# Mostrar Resumen Global
# ------------------------------------------------------------
def mostrar_resumen_global(data_enriched, max_pdfs=5):
    """
    Muestra un resumen global del procesamiento de PDFs.

    Args:
        data_enriched (dict): Datos procesados enriquecidos.
        max_pdfs (int): Número máximo de PDFs a mostrar en el resumen.
    """
    print("\n--- Resumen Global del Procesamiento ---")
    for idx, (pdf_path, pdf_data) in enumerate(data_enriched.items(), start=1):
        total_paginas = len(pdf_data.get('paginas', []))
        total_capitulos = sum(
            1 for p in pdf_data['paginas'] for c in p['contenido']
            if c['tipo'] == 'texto' and 'CAPÍTULO' in c['texto']
        )
        total_tablas = sum(
            1 for p in pdf_data['paginas'] for c in p['contenido']
            if c['tipo'] == 'tabla'
        )

        print(f"\nArchivo {idx}: {os.path.basename(pdf_path)}")
        print(f"  Total de páginas procesadas: {total_paginas}")
        print(f"  Total de capítulos detectados: {total_capitulos}")
        print(f"  Total de tablas detectadas: {total_tablas}")

        if idx >= max_pdfs:
            print("... (se han procesado más archivos, pero se limitan los resultados mostrados)")
            break


# Llama a la función con un límite de 5 PDFs para mostrar
mostrar_resumen_global(data_enriched, max_pdfs=5)

In [ ]:
from loaders.metadata_verification import verificar_metadatos_guardados

# ------------------------------------------------------------
# Verificar metadatos guardados con límites
# ------------------------------------------------------------
print("\n--- Verificando Metadatos Guardados ---")
verificar_metadatos_guardados(data_enriched, max_files=3, max_pages=2, max_blocks=5)

In [ ]:
# ------------------------------------------------------------
# Importar módulos necesarios
# ------------------------------------------------------------
from loaders.review_tools import revisar_estructuras_generadas, revisar_paginas_especificas

# ------------------------------------------------------------
# Revisar ejemplos de estructuras generadas
# ------------------------------------------------------------
print("\n--- Revisando ejemplos de estructuras generadas ---")
revisar_estructuras_generadas(data_enriched, num_ejemplos=3)

In [ ]:
# ------------------------------------------------------------
# Revisar páginas específicas
# ------------------------------------------------------------
print("\n--- Revisando páginas específicas del archivo seleccionado ---")
revisar_paginas_especificas(data_enriched, "capitulo13", paginas_deseadas=list(range(6, 11)))

# Chunking

In [13]:
# ------------------------------------------------------------
# Importar módulos necesarios
# ------------------------------------------------------------
from chunking.separated_chunker import chunk_documents_separately
from chunking.chunk_verifier import verificar_chunks_narrativos_completos, verificar_chunks_tablas_completos
from chunking.chunk_analysis import analizar_chunks_narrativos, analizar_chunks_tablas, calcular_estadisticas_chunks_por_capitulo

In [ ]:
# ------------------------------------------------------------
# Validación de Datos Enriquecidos
# ------------------------------------------------------------
print("\n--- Verificando Datos Enriquecidos ---")
print(f"Total de archivos procesados: {len(data_enriched)}")
for archivo, contenido in data_enriched.items():
    print(f"Archivo: {archivo}, Total de páginas: {len(contenido['paginas'])}")

In [ ]:
# ------------------------------------------------------------
# Generar Chunks Separados
# ------------------------------------------------------------
print("\n--- Generando Chunks Narrativos y de Tablas ---")
chunks_narrativos, chunks_tablas = chunk_documents_separately(data_enriched, chunk_size=3000)

# Verificar resultados
print(f"Total de chunks narrativos: {len(chunks_narrativos)}")
print(f"Total de chunks de tablas: {len(chunks_tablas)}")

In [ ]:
# ------------------------------------------------------------
# Verificar Ejemplos de Chunks
# ------------------------------------------------------------
verificar_chunks_narrativos_completos(chunks_narrativos, num_chunks=3)
verificar_chunks_tablas_completos(chunks_tablas, num_chunks=3)

In [ ]:
# ------------------------------------------------------------
# Analizar Estadísticas Generales de los Chunks
# ------------------------------------------------------------
print("\n--- Analizando Estadísticas de los Chunks Narrativos ---")
estadisticas_narrativos = analizar_chunks_narrativos(chunks_narrativos)
print(f"  Total de chunks: {estadisticas_narrativos['total_chunks']}")
print(f"  Total de caracteres: {estadisticas_narrativos['total_caracteres']}")
print(f"  Promedio de caracteres por chunk: {estadisticas_narrativos['promedio_caracteres']:.2f}")

print("\n--- Analizando Estadísticas de los Chunks de Tablas ---")
estadisticas_tablas = analizar_chunks_tablas(chunks_tablas)
print(f"  Total de chunks de tablas: {estadisticas_tablas['total_chunks']}")
print(f"  Total de filas: {estadisticas_tablas['total_filas']}")
print(f"  Promedio de filas por tabla: {estadisticas_tablas['promedio_filas']:.2f}")
print(f"  Promedio de columnas por tabla: {estadisticas_tablas['promedio_columnas']:.2f}")

In [ ]:
import matplotlib.pyplot as plt

# ------------------------------------------------------------
# Histograma para Chunks Narrativos (Tamaño en Caracteres)
# ------------------------------------------------------------
def generar_histograma_narrativos(chunks_narrativos):
    tamaños = [len(chunk["texto"]) for chunk in chunks_narrativos]

    plt.figure(figsize=(10, 6))
    plt.hist(tamaños, bins=30, edgecolor='black', alpha=0.7)
    plt.title("Distribución del Tamaño de Chunks Narrativos")
    plt.xlabel("Tamaño del Chunk (caracteres)")
    plt.ylabel("Cantidad de Chunks")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

# ------------------------------------------------------------
# Histograma para Chunks de Tablas (Filas y Columnas)
# ------------------------------------------------------------
def generar_histograma_tablas(chunks_tablas):
    filas = [chunk["metadatos"]["filas"] for chunk in chunks_tablas]
    columnas = [chunk["metadatos"]["columnas"] for chunk in chunks_tablas]

    # Histograma de Filas
    plt.figure(figsize=(10, 6))
    plt.hist(filas, bins=20, color='orange', edgecolor='black', alpha=0.7)
    plt.title("Distribución del Número de Filas en Tablas")
    plt.xlabel("Cantidad de Filas por Tabla")
    plt.ylabel("Cantidad de Tablas")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

    # Histograma de Columnas
    plt.figure(figsize=(10, 6))
    plt.hist(columnas, bins=20, color='green', edgecolor='black', alpha=0.7)
    plt.title("Distribución del Número de Columnas en Tablas")
    plt.xlabel("Cantidad de Columnas por Tabla")
    plt.ylabel("Cantidad de Tablas")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

# ------------------------------------------------------------
# Generar Histogramas
# ------------------------------------------------------------
print("\n--- Generando Histogramas para Chunks Narrativos ---")
generar_histograma_narrativos(chunks_narrativos)

print("\n--- Generando Histogramas para Chunks de Tablas ---")
generar_histograma_tablas(chunks_tablas)

# Embeddings

In [ ]:
# Importar dependencias
from embedding.embedding_v2 import generate_embeddings_v2
from vector_store_client.vector_store_client_v2 import insert_embeddings_v2
from qdrant_client import QdrantClient
from tqdm import tqdm

In [ ]:
# ------------------------------------------------------------
# Preparar los Chunks para Generar Embeddings
# ------------------------------------------------------------
# Combinar todos los chunks narrativos y de tablas en un solo formato estándar
all_chunks = []

for chunk in chunks_narrativos:
    all_chunks.append({
        "type": "narrative",
        "content": chunk["texto"],
        "metadata": chunk["metadatos"]
    })

for chunk in chunks_tablas:
    tabla_contenido = "\n".join(["\t".join(map(str, row)) for row in chunk["tabla"]])
    all_chunks.append({
        "type": "table",
        "content": tabla_contenido,
        "metadata": chunk["metadatos"]
    })

print(f"Total de chunks preparados para embeddings: {len(all_chunks)}")

In [ ]:
# ------------------------------------------------------------
# Generar Embeddings y Metadatos
# ------------------------------------------------------------
from embedding.embedding_v2 import generate_embeddings_v2

all_embeddings = generate_embeddings_v2(all_chunks)
print(f"Total de embeddings generados: {len(all_embeddings)}")

In [ ]:
# Separar embeddings y metadatos en listas separadas
embeddings_list = []
metadata_list = []

# Generar embeddings y metadatos con validaciones
for idx, entry in enumerate(all_embeddings):
    try:
        # Validar estructura esperada
        if "vector" in entry and "default" in entry["vector"]:
            embedding = entry["vector"]["default"]
        else:
            raise KeyError(f"Claves faltantes en vector en índice {idx}: {entry}")

        if "payload" in entry:
            payload = entry["payload"]
        else:
            raise KeyError(f"Clave 'payload' faltante en índice {idx}: {entry}")

        # Validar longitud del embedding
        if len(embedding) != 384:
            raise ValueError(f"Embedding en índice {idx} tiene tamaño incorrecto: {len(embedding)}")

        # Agregar a las listas
        embeddings_list.append(embedding)
        metadata_list.append(payload)
    except KeyError as e:
        print(f"Error procesando embedding en el índice {idx}: {e}")
    except Exception as e:
        print(f"Error inesperado en el índice {idx}: {e}")

# Validar resultados
print(f"Total embeddings después del bucle: {len(embeddings_list)}")
print(f"Ejemplo de embedding: {embeddings_list[0] if embeddings_list else 'Ninguno'}")
print(f"Total metadatos después del bucle: {len(metadata_list)}")
print(f"Ejemplo de metadata: {metadata_list[0] if metadata_list else 'Ninguno'}")

In [ ]:
print(f"embeddings_list: {len(embeddings_list) if embeddings_list else 'None'}")
print(f"metadata_list: {len(metadata_list) if metadata_list else 'None'}")

In [24]:
if any(len(embedding) != 384 for embedding in embeddings_list):
    print("Error: Algunos embeddings no tienen la dimensión adecuada.")

if any(not isinstance(metadata, dict) for metadata in metadata_list):
    print("Error: Algunos metadatos no tienen el formato adecuado.")

In [ ]:
print("Ejemplo de embedding:", embeddings_list[0])
print("Ejemplo de metadato:", metadata_list[0])

In [ ]:
# Verificar detalles de la colección
collection_info = qdrant_client.get_collection(INDEX_NAME)
print(collection_info)

In [ ]:
for idx, (embedding, metadata) in enumerate(zip(embeddings_list, metadata_list)):
    point = {
        "id": idx,
        "vector": {"default": embedding},  # Clave 'default' requerida por Qdrant
        "payload": metadata
    }
    print(f"Punto {idx}: {point}")
    if idx == 0:  # Ver solo el primer punto para simplificar
        break

In [ ]:
test_point = {
    "id": 1,
    "vector": {"default": embeddings_list[0]},  # Embedding bajo el nombre 'default'
    "payload": metadata_list[0]
}

response = qdrant_client.upsert(
    collection_name="embeddings-version-2",  # Nombre correcto de la colección
    points=[test_point]
)

print("Respuesta de inserción manual:", response)

In [ ]:
print(type(embeddings_list[0]))
print(len(embeddings_list[0]))

In [ ]:
batch = [
    {
        "id": i,
        "vector": {"default": embeddings_list[i]},
        "payload": metadata_list[i]
    }
    for i in range(10)  # Prueba con los primeros 10 puntos
]

response = qdrant_client.upsert(
    collection_name="embeddings-version-2",
    points=batch
)
print("Respuesta de inserción de lote pequeño:", response)

In [ ]:
# Verificar datos antes de insertar
if embeddings_list and metadata_list:
    print(f"Ejemplo de embedding: {embeddings_list[0][:5]}... (Tamaño: {len(embeddings_list[0])})")
    print(f"Ejemplo de payload: {metadata_list[0]}")

    print(f"Insertando {len(embeddings_list)} embeddings en la colección 'embeddings-version-2' en lotes de 100...")
    insert_embeddings_v2(
        client=qdrant_client,
        index_name="embeddings-version-2",
        embeddings=embeddings_list,
        metadata_list=metadata_list,
        batch_size=100
    )
    print("Embeddings generados e insertados en Qdrant con éxito.")
else:
    print("No hay embeddings o metadatos para insertar.")

In [ ]:
# ------------------------------------------------------------
# Validar Resultados
# ------------------------------------------------------------
print("\n--- Validación de Embeddings ---")
if all_embeddings:
    num_valid_embeddings = len(embeddings_list)
    print(f"Embeddings válidos: {num_valid_embeddings}/{len(all_chunks)}")
    print("Ejemplo de embedding:")
    print(f"Tipo: {metadata_list[0]['type']}")
    print(f"Contenido: {metadata_list[0]['content'][:100]}...")
    print(f"Embedding: {embeddings_list[0][:5]}...")  # Mostrar primeros valores
else:
    print("No se generaron embeddings.")

# Retrieval

In [61]:
from embedding.embedding_v2 import generate_query_embedding
from vector_store_client.vector_store_search import search_embeddings, format_search_results
from vector_store_client.vector_store_client import search_qdrant

# ------------------------------------------------------------
# Definir la consulta y generar su embedding
# ------------------------------------------------------------
query = "¿Qué se entiende por cerveza?"

# Generar embedding para la consulta
try:
    query_embedding = generate_query_embedding(query)

    # Validar el formato del embedding generado
    if not isinstance(query_embedding, list) or len(query_embedding) != 384:
        raise ValueError("El embedding de la consulta no tiene el formato esperado (lista con 384 dimensiones).")
    
    print(f"Tipo de query_embedding: {type(query_embedding)}")
    print(f"Tamaño de query_embedding: {len(query_embedding)}")
    print(f"Primeros valores: {query_embedding[:5]}")
except Exception as e:
    print(f"Error al generar el embedding de la consulta: {e}")
    raise

# ------------------------------------------------------------
# Realizar la búsqueda en Qdrant
# ------------------------------------------------------------
index_name = "embeddings-version-2"  # Nombre de la colección en Qdrant
limit = 50  # Número de resultados a devolver

try:
    print(f"Realizando búsqueda en la colección '{index_name}'...")
    results = search_qdrant(qdrant_client, index_name, query_embedding, limit=limit)
except Exception as e:
    print(f"Error al realizar la búsqueda en Qdrant: {e}")
    results = []

# ------------------------------------------------------------
# Formatear y mostrar los resultados
# ------------------------------------------------------------
if results:
    format_search_results(query, results)
else:
    print("No se encontraron resultados para la consulta.")

Generando embeddings para 1 chunks...


Generando embeddings: 100%|██████████| 1/1 [00:00<00:00, 66.09chunk/s]

Total de embeddings generados: 1
Tipo de query_embedding: <class 'list'>
Tamaño de query_embedding: 384
Primeros valores: [-0.014241083525121212, 0.03439000993967056, -0.04755904898047447, -0.006681044586002827, -0.06351680308580399]
Realizando búsqueda en la colección 'embeddings-version-2'...



Resultados de la búsqueda para la consulta: '¿Qué se entiende por cerveza?'
Total de resultados: 50
--------------------------------------------------------------------------------
Resultado 1:
ID: 2078
Score: 0.5312575
Contenido del payload:
  type: table
  content: Definir el objeto de muestreo	¿Qué tipo de alimento se va a muestrear?
  content_length: 70
--------------------------------------------------------------------------------
Resultado 2:
ID: 2066
Score: 0.5017834
Contenido del payload:
  type: table
  content: ACIÓN Y
REGISTROS	durante el tiempo necesario en cada caso.
  content_length: 59
--------------------------------------------------------------------------------
Resultado 3:
ID: 1175
Score: 0.4983374
Contenido del payload:
  type: narrative
  content: 11. MUESTREO.
Se seguirán los procedimientos recomendados en la Norma FIL 50B: 1985.
RESOLUCIÓN GMC 
  content_length: 3002
--------------------------------------------------------------------------------
Resultado 4:


In [78]:
print(results[0].payload['content'])

Definir el objeto de muestreo	¿Qué tipo de alimento se va a muestrear?


In [65]:
chunks_basic = [point.payload['content'] for point in results]

In [66]:
print("\n---\n".join(chunks_basic))

Definir el objeto de muestreo	¿Qué tipo de alimento se va a muestrear?
---
ACIÓN Y
REGISTROS	durante el tiempo necesario en cada caso.
---
11. MUESTREO.
Se seguirán los procedimientos recomendados en la Norma FIL 50B: 1985.
RESOLUCIÓN GMC N° 032/96
Incorporada por Resolución MSyAS N° 435 del 26.06.97
Se deroga toda legislación del Código Alimentario Argentino que se oponga al dictado de la
presente Resolución
REGLAMENTO TÉCNICO MERCOSUR DE IDENTIDAD Y CALIDAD DEL QUESO
TILSIT
Art 1° - Aprobar el Reglamento Técnico MERCOSUR de Identidad y Calidad del Queso Tilsit que
figura en el Anexo de la presente Resolución.
Art 2° - Los Estados Partes pondrán en vigencia las disposiciones legislativas, reglamentarias y
administrativas necesarias para dar cumplimiento a la presente Resolución a través de los
siguientes organismos:
Argentina: Ministerio de Salud y Acción Social; Ministerio de Economía y Obras y Servicios
Públicos; Secretaría de Agricultura, Pesca y Alimentación; (Servicio Nacional de

In [67]:
import retrievers.summarization as summ

chunks_adv = summ.summarize_multiple_chunks(query,chunks_basic)

Query: ¿Qué se entiende por cerveza?


Processing chunks:   6%|▌         | 3/50 [00:02<00:36,  1.28it/s]

Relevant Summary | Con el nombre de Queso Tilsit se entiende el queso madurado que se obtiene por coagulación de la leche por medio del cuajo y/u otras enzimas coagulantes apropiadas, complementada o no por la acción de bacterias lácticas específicas.


Processing chunks:  20%|██        | 10/50 [00:07<00:37,  1.07it/s]

Relevant Summary | Se entiende por crema de leche a granel de uso industrial como la crema transportada en volumen de un establecimiento industrializador de productos lácteos a otro, a ser procesada y que no sea destinada directamente al consumidor final.


Processing chunks:  50%|█████     | 25/50 [00:15<00:17,  1.46it/s]

Relevant Summary | Con el nombre de Queso Pategrás Sandwich se entiende el queso madurado que se obtiene por coagulación de la leche por medio del cuajo y/u otras enzimas coagulantes apropiadas, complementada o no por la acción de bacterias lácticas específicas.


Processing chunks:  92%|█████████▏| 46/50 [00:26<00:03,  1.22it/s]

Relevant Summary | La cerveza se entiende como un producto en polvo que, al dispersarse en agua y/o leche, con o sin el agregado de edulcorantes nutritivos, huevos y/o yemas, y por posterior congelamiento, permite la obtención casera de postres para consumir helados. Los ingredientes permitidos para elaborar estos polvos para preparar postres incluyen almidones, grasas comestibles, leche en polvo, edulcorantes nutritivos, huevos deshidratados, crema de leche en polvo, dulce de leche, frutas secas, productos fruitivos como cacao, chocolate, café, malta, y granos o semillas. Además, se permite el agregado de ciertos aditivos autorizados por el código de alimentos.


Processing chunks: 100%|██████████| 50/50 [00:28<00:00,  1.75it/s]


In [68]:
chunks_basic[46]

'- otorgar al procedimiento de retiro el número identificatorio único de retiro (NIUR) que\nconstará de seis (6) dígitos, los cuatros (4) primeros corresponden a números consecutivos\ny los dos (2) últimos dígitos (separados por un guión) identificarán el año\n- notificar a las ASP y coordinar con ellas el retiro en todo el territorio del país\n- evaluar la información requerida a la EA y la estrategia de retiro propuesta y, en caso de\nconsiderarlo necesario, indicará cambios y formulará las observaciones en un plazo no\nmayor a 5 días de presentada la documentación.\n- comunicar, cuando sea necesario, a otras autoridades, organismos nacionales e\ninternacionales.\n- realizar el seguimiento a nivel nacional de la acción de retiro,\n- implementar las verificaciones del progreso del retiro en caso de haber detectado el\nincidente\n- supervisar la disposición y el destino de los productos importados recuperados en el país.\n- informar a las ASP y a la Ciudad Autónoma de Buenos Aires sobr

In [69]:
chunks_basic = chunks_basic[:len(chunks_adv)]

In [60]:
import answering.answering as ans

In [70]:
answer_basic = ans.answer_query_with_context(query,chunks_basic)
answer_basic

'Respuesta: En base al contexto proporcionado, se entiende por cerveza una bebida alcohólica que se obtiene a través de la fermentación de ingredientes como la cebada, el lúpulo, el agua y la levadura. La cerveza es una bebida popular consumida en muchos países y puede variar en sabor, color y contenido alcohólico dependiendo de los ingredientes y el proceso de elaboración utilizado.'

In [71]:
answer_adv = ans.answer_query_with_context(query,chunks_adv)
answer_adv

'La cerveza se entiende como una bebida alcohólica obtenida a través de la fermentación de ingredientes como la cebada, el lúpulo, el agua y la levadura. No debe confundirse con el producto en polvo mencionado en el contexto, que al dispersarse en agua y/o leche permite la obtención de postres helados.'

In [74]:
import evaluation.evaluation_ragas as eval

In [75]:
ground_truth = '''Se entiende exclusivamente por cerveza la bebida resultante de fermentar, mediante
levadura cervecera, al mosto de cebada malteada o de extracto de malta, sometido
previamente a un proceso de cocción, adicionado de lúpulo. Una parte de la cebada
malteada o de extracto de malta podrá ser reemplazada por adjuntos cerveceros.
La cerveza negra podrá ser azucarada.
La cerveza podrá ser adicionada de colorantes, saborizantes y aromatizantes.'''


ragas_basic = eval.evaluate_ragas(query,answer_basic, chunks_basic, ground_truth)
ragas_basic

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,context_precision,faithfulness,answer_relevancy,context_recall
0,¿Qué se entiende por cerveza?,[Definir el objeto de muestreo\t¿Qué tipo de a...,"Respuesta: En base al contexto proporcionado, ...",Se entiende exclusivamente por cerveza la bebi...,0.0,0.0,0.877575,0.0


In [77]:
ragas_adv = eval.evaluate_ragas(query,answer_adv, chunks_adv, ground_truth)
ragas_adv

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,context_precision,faithfulness,answer_relevancy,context_recall
0,¿Qué se entiende por cerveza?,[Con el nombre de Queso Tilsit se entiende el ...,La cerveza se entiende como una bebida alcohól...,Se entiende exclusivamente por cerveza la bebi...,0.0,0.5,0.885718,0.0


In [53]:
from datasets import Dataset 
import os
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, LLMContextRecall
import os

# Set the OpenAI API key
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

def evaluate_ragas(query, answer,chunks, ground_truth):
    data = {
        'question': [
            query
        ],
        'response': [
                answer
        ],
        'contexts': [
                chunks
        ],
        'ground_truth': [
            ground_truth
        ]
    }

    dataset = Dataset.from_dict(data)

    context_recall = LLMContextRecall()
    score = evaluate(dataset, metrics=[context_precision, faithfulness, answer_relevancy, context_recall])
    df = score.to_pandas()
    return df

In [ ]:
ground_truth = '''Se considera polen no apto para el consumo, aquel que presente una o más de las siguientes
    características:
    1. Caracteres organolépticos anormales
    2. Exceso de polvillo o de propóleos
    3. Anormalidades en la observación microscópica
    4. Composición analítica diferente a la consignada anteriormente
    5. Características microbiológicas superiores a los límites establecidos
    6. Ataque de insectos, parásitos o sus larvas
    7. Residuos de plaguicidas
    8. Substancias conservadoras
    9. Impurezas no retenidas por un tamiz IRAM 500 μ (N° 35) más de 5 por 1000.'''


df_ragas = evaluate_ragas(query,answer_adv, chunks_adv, ground_truth)
df_ragas

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,context_precision,faithfulness,answer_relevancy,context_recall
0,¿Qué se dice en la normativa respecto al polen?,[El Reglamento Técnico MERCOSUR de Identidad y...,En la normativa se establecen las característi...,"Se considera polen no apto para el consumo, aq...",0.5,1.0,0.807211,0.5


In [59]:
df_ragas = evaluate_ragas(query,answer_basic, chunks_basic, ground_truth)
df_ragas

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,context_precision,faithfulness,answer_relevancy,context_recall
0,¿Qué se dice en la normativa respecto al polen?,[Definir el objeto de muestreo\t¿Qué tipo de a...,"En la normativa alimentaria, el polen puede es...","Se considera polen no apto para el consumo, aq...",0.0,0.4,0.0,0.0


In [ ]:
answer_adv = answer_query_with_context(query,chunks_adv)
answer_adv

'En la normativa se establecen las características que debe cumplir el polen para ser considerado apto para el consumo humano. Debe estar limpio, seco, sin restos de insectos, larvas o huevos, ni exceso de propóleos, y presentar un olor característico de acuerdo a la especie floral que provenga. Además, se detallan ciertas características analíticas de composición que debe cumplir, como porcentaje de humedad, cenizas, proteínas, pH y hidratos de carbono. En cuanto a las características microbiológicas, se establecen límites para los gérmenes aerobios no patógenos, hongos y se exige la ausencia de gérmenes patógenos. El polen debe envasarse en recipientes bromatológicamente aptos de hasta 250 g, con cierre que impida la absorción de humedad, siendo los envases de vidrio o plástico rígido transparente. Se detallan también las características que hacen que el polen no sea apto para el consumo, como anormalidades organolépticas, exceso de polvillo o propóleos, entre otros. Finalmente, se e

In [58]:
answer_basic = answer_query_with_context(query,chunks_basic)
answer_basic

'En la normativa alimentaria, el polen puede estar regulado en cuanto a su presencia en los alimentos, especialmente si se trata de alérgenos que deben ser declarados en el etiquetado de los productos. Además, es importante tener en cuenta que en el contexto proporcionado no se hace mención específica al polen, por lo que no se puede brindar información detallada sobre regulaciones específicas relacionadas con este componente en particular.'